In [8]:
import pandas as pd
import numpy as np
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models import Word2Vec
import nltk

df = pd.read_csv('네이버웹툰_최종_프로젝트.csv')
print(df.shape)

dataset = pd.read_csv('movies04293.csv')

tokenized_sentences = [word_tokenize(sentence) for sentence in dataset['plot']]

model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, sg=0)
model.save("word2vec_model")
Model = Word2Vec.load("word2vec_model")

(2470, 13)


In [9]:
# Load the pre-trained Word2Vec model
Model = Word2Vec.load("word2vec_model")

df = pd.read_csv('네이버웹툰_최종_프로젝트.csv')

#숫자랑 문자 제거
pattern = r'[^a-zA-Z가-힣]'
df['줄거리'] = df['줄거리'].apply(lambda x: re.sub(pattern=pattern, repl=' ', string=x))

#토큰화
web_tokenized_sentences = [word_tokenize(sentence) for sentence in df['줄거리']]

similarities = []
for i in range(len(df)):
    similarities_row = []
    for j in range(len(df)):
        if i == j:
            similarity_score = 1.0  # 같은 행의 경우 유사도는 1.0
        else:
            sentence1 = web_tokenized_sentences[i]
            sentence2 = web_tokenized_sentences[j]
            # Calculate cosine similarity between Word2Vec embeddings
            similarity_score = Model.wv.n_similarity(sentence1, sentence2)
        similarities_row.append(similarity_score)
    similarities.append(similarities_row)

# 결과 출력 (similarity[i][j]는 웹툰 i와 웹툰 j 간의 유사도)
for i in range(len(similarities)):
    for j in range(len(similarities[i])):
        print(f"Similarity between 웹툰 {i+1} and 웹툰 {j+1}: {similarities[i][j]}")

KeyboardInterrupt: 

In [ ]:
def find_similar_sentences(title, top_n=5):
    title_data = df[df['제목'].str.contains(title, case=True, na=False)]
    title_index = title_data.index.values[0]

    # 해당 작품과 다른 작품 간의 유사도
    title_similarity_scores = similarities[title_index]

    # 유사도가 높은 상위 N개 인덱스를 찾음
    similar_indexes = np.argsort(title_similarity_scores)[::-1][:top_n]

    # 유사도 값 가져오기
    similar_cosine_values = [title_similarity_scores[i] for i in similar_indexes]

    # 유사한 작품 데이터 프레임 생성
    similar_titles = df.iloc[similar_indexes]

    # "유사도" 열 추가
    similar_titles['유사도'] = similar_cosine_values

    return similar_titles

In [ ]:
find_similar_sentences('참교육',top_n=5)